In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
config = {
    'domain': 'drink',
    'debug': True,
    'postgres': False,
    'parallelism': 1,
    'splits': [0,1,2],
    'disc_model_class': 'logreg',
    'supervision': 'generative',
}

In [29]:
# Get DB connection string and add to globals
# NOTE: $SNORKELDB must be set before any snorkel imports
import os

default_db_name = 'babble_' + config['domain'] + ('_debug' if config.get('debug', False) else '')
DB_NAME = config.get('db_name', default_db_name)
if 'postgres' in config and config['postgres']:
    DB_TYPE = 'postgres'
else:
    DB_TYPE = 'sqlite'
    DB_NAME += '.db'
DB_ADDR = "localhost:{0}".format(config['db_port']) if 'db_port' in config else ""
os.environ['SNORKELDB'] = '{0}://{1}/{2}'.format(DB_TYPE, DB_ADDR, DB_NAME)
print("$SNORKELDB = {0}".format(os.environ['SNORKELDB']))

$SNORKELDB = sqlite:///babble_drink_debug.db


In [30]:
from snorkel import SnorkelSession
session = SnorkelSession()

# Resolve config conflicts (nb_config > local_config > global_config)
from snorkel.contrib.babble.pipelines import merge_configs, get_local_pipeline
config = merge_configs(config)

if config['debug']:
    print("NOTE: --debug=True: modifying parameters...")
    config['max_docs'] = 100
    config['gen_model_search_space'] = 2
    config['disc_model_search_space'] = 2
    config['gen_params_default']['epochs'] = 25
    config['disc_params_default']['n_epochs'] = 5

from snorkel.models import candidate_subclass
candidate_class = candidate_subclass(config['candidate_name'], config['candidate_entities'])

pipeline = get_local_pipeline(config['domain'])
pipe = pipeline(session, candidate_class, config)

Overwriting splits=[0, 1] to splits=[0, 1, 2]
Overwriting disc_model_class=inception_v3 to disc_model_class=logreg
Overwriting epochs=25 to epochs=100
Overwriting disc_model_class=inception_v3 to disc_model_class=logreg
Overwriting supervision=traditional to supervision=generative
Overwriting gen_model_search_space=2 to gen_model_search_space=1
NOTE: --debug=True: modifying parameters...


In [31]:
%time pipe.parse()

Clearing existing...
Running UDF...
Running UDF...
CPU times: user 20.6 s, sys: 568 ms, total: 21.2 s
Wall time: 21.7 s


In [32]:
%time pipe.extract()

Extraction was performed during parse stage.
Candidates [Split 0]: 15711
Candidates [Split 1]: 3377
Candidates [Split 2]: 0
CPU times: user 16 ms, sys: 16 ms, total: 32 ms
Wall time: 18 ms


In [33]:
%time pipe.load_gold()

Reading train CSV!
Num HITs unique: 1525
Num HITs total: 3050
Unanimous: 4542
Majority: 2211
Bad: 450
Reading val CSV!
Num HITs unique: 184
Num HITs total: 368
Unanimous: 474
Majority: 318
Bad: 100
AnnotatorLabels created: 5648
AnnotatorLabels created: 634
CPU times: user 1min, sys: 692 ms, total: 1min 1s
Wall time: 1min 1s


In [34]:
# %time pipe.featurize()

In [ ]:
%time pipe.collect()

Reading train CSV!
Num HITs unique: 44
Num HITs total: 132
Unanimous: 112
Majority: 54
Bad: 37
Building list of target candidate ids...
Collected 139 unique target candidate ids from 370 explanations.
Gathering desired candidates...
Found 139/139 desired candidates
Linking explanations to candidates...
Linked 370/370 explanations
Calling babbler...
Flushing all parses from previous explanation set.
Created grammar with 453 rules
121 explanation(s) out of 370 were parseable.
171 parse(s) generated from 370 explanation(s).
49 parse(s) remain (122 parse(s) removed by DuplicateSemanticsFilter).
24 parse(s) remain (25 parse(s) removed by ConsistencyFilter).
### Applying labeling functions to split 1
[========================================] 100%

### Done in 4.8s.

18 parse(s) remain (6 parse(s) removed by UniformSignatureFilter: (0 None, 6 All)).
14 parse(s) remain (4 parse(s) removed by DuplicateSignatureFilter).
Added 14 parse(s) from 13 explanations to set. (Total # parses = 14)
CPU ti

In [ ]:
%time pipe.label()

Clearing existing...
Running UDF...
[========================================] 100%



In [ ]:
%time pipe.supervise()

In [ ]:
config['display_marginals'] = False
config['download_data'] = False
config['disc_model_class'] = 'inception_v3'

%time pipe.classify(slim_ws_path='/dfs/scratch0/paroma/slim_new/slim_ws/')

Drink:
208, 416, 624, 858